In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE65986"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE65986"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE65986.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE65986.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE65986.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrated copy number and expression analysis identifies profiles of whole-arm chromosomal alterations and subgroups with favorable outcome in ovarian clear cell carcinomas"
!Series_summary	"Ovarian clear cell carcinoma (CCC) is generally associated with chemoresistance and poor clinical outcome, even with early diagnosis; whereas high-grade serous carcinomas (SCs) and endometrioid carcinomas (ECs) are commonly chemosensitive at advanced stages. Although an integrated genomic analysis of SC has been performed, conclusive views on copy number and expression profiles for CCC are still limited. In this study, we performed single nucleotide polymorphism arrays in 57 (31 CCCs, 14 SCs, and 12 ECs) and expression microarrays in 55 epithelial ovarian cancers (25 CCCs, 16 SCs, and 14 ECs), and then evaluated PIK3CA mutations and ARID1A expression in CCCs. SNP array analysis classified 13% of CCCs into a cluster with high frequency and focal range of copy 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# analyzed by Affymetrix U133plus2 array
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary

# Trait: Endometrioid Cancer
# We can find 'histology: Endometrioid' in key 0
trait_row = 0

# Age: Available in key 1
age_row = 1

# Gender: Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert histology data to binary (Endometrioid = 1, others = 0)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if it's related to the trait of interest
    if 'endometrioid' in value.lower():
        return 1
    elif 'clear' in value.lower() or 'serous' in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age data to continuous numeric values"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender data (Not applicable for this dataset)"""
    return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Perform initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Get clinical data using the geo_select_clinical_features function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM1612097': [0.0, 64.0], 'GSM1612098': [0.0, 57.0], 'GSM1612099': [0.0, 59.0], 'GSM1612100': [0.0, 50.0], 'GSM1612101': [0.0, 52.0], 'GSM1612102': [0.0, 66.0], 'GSM1612103': [0.0, 67.0], 'GSM1612104': [0.0, 37.0], 'GSM1612105': [0.0, 53.0], 'GSM1612106': [0.0, 46.0], 'GSM1612107': [0.0, 57.0], 'GSM1612108': [0.0, 51.0], 'GSM1612109': [0.0, 55.0], 'GSM1612110': [0.0, 39.0], 'GSM1612111': [0.0, 71.0], 'GSM1612112': [0.0, 54.0], 'GSM1612113': [0.0, 64.0], 'GSM1612114': [0.0, 53.0], 'GSM1612115': [0.0, 45.0], 'GSM1612116': [0.0, 80.0], 'GSM1612117': [0.0, 55.0], 'GSM1612118': [0.0, 64.0], 'GSM1612119': [0.0, 74.0], 'GSM1612120': [0.0, 67.0], 'GSM1612121': [0.0, 39.0], 'GSM1612122': [1.0, 43.0], 'GSM1612123': [1.0, 39.0], 'GSM1612124': [1.0, 49.0], 'GSM1612125': [1.0, 61.0], 'GSM1612126': [1.0, 64.0], 'GSM1612127': [1.0, 61.0], 'GSM1612128': [1.0, 32.0], 'GSM1612129': [1.0, 69.0], 'GSM1612130': [1.0, 45.0], 'GSM1612131': [1.0, 52.0], 'GSM1612132': [1.0, 74.0], '

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 80
Header line: "ID_REF"	"GSM1612097"	"GSM1612098"	"GSM1612099"	"GSM1612100"	"GSM1612101"	"GSM1612102"	"GSM1612103"	"GSM1612104"	"GSM1612105"	"GSM1612106"	"GSM1612107"	"GSM1612108"	"GSM1612109"	"GSM1612110"	"GSM1612111"	"GSM1612112"	"GSM1612113"	"GSM1612114"	"GSM1612115"	"GSM1612116"	"GSM1612117"	"GSM1612118"	"GSM1612119"	"GSM1612120"	"GSM1612121"	"GSM1612122"	"GSM1612123"	"GSM1612124"	"GSM1612125"	"GSM1612126"	"GSM1612127"	"GSM1612128"	"GSM1612129"	"GSM1612130"	"GSM1612131"	"GSM1612132"	"GSM1612133"	"GSM1612134"	"GSM1612135"	"GSM1612136"	"GSM1612137"	"GSM1612138"	"GSM1612139"	"GSM1612140"	"GSM1612141"	"GSM1612142"	"GSM1612143"	"GSM1612144"	"GSM1612145"	"GSM1612146"	"GSM1612147"	"GSM1612148"	"GSM1612149"	"GSM1612150"	"GSM1612151"
First data line: "1007_s_at"	852.9	1071.9	542.4	325	770.2	534.5	542.9	1571.7	1062.4	1419.2	428.2	473	760.9	1002.9	252.4	256.9	586.1	318.1	314.9	304.3	580.9	430.3	446.9	727.4	406.2	1041.9	291.1	857.5	1225.7	723.1	292.8	563.4	449.4	362.

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the provided data appear to be Affymetrix probe IDs (e.g., "1007_s_at", "1053_at")
# These are not standard human gene symbols and will need to be mapped to gene symbols
# Affymetrix probe IDs need to be converted to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:


Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE65986
Line 6: !Series_title = Integrated copy number and expression analysis identifies profiles of whole-arm chromosomal alterations and subgroups with favorable outcome in ovarian clear cell carcinomas
Line 7: !Series_geo_accession = GSE65986
Line 8: !Series_status = Public on May 11 2015
Line 9: !Series_submission_date = Feb 17 2015
Line 10: !Series_last_update_date = Jul 12 2019
Line 11: !Series_pubmed_id = 26147301
Line 12: !Series_pubmed_id = 27659536
Line 13: !Series_summary = Ovarian clear cell carcinoma (CCC) is generally associated with chemoresistance and poor clinical outcome, even with early diagnosis; whereas high-grade serous carcinomas (SCs) and endometrioid carcinomas (ECs) are commonly chemosensitive at advanced stag


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze column names to find gene identifier and gene symbol columns
# From the previewed gene annotation, we can see:
# - The gene identifiers (Affymetrix probe IDs) are stored in the 'ID' column
# - The gene symbols are stored in the 'Gene Symbol' column

# 2. Extract gene mapping data
gene_mapping_df = gene_annotation.loc[:, ['ID', 'Gene Symbol']]
gene_mapping_df = gene_mapping_df.rename(columns={'Gene Symbol': 'Gene'})

# Check the first few rows of the mapping
print("Gene mapping preview (first 5 rows):")
print(gene_mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene expression data
# Use the apply_gene_mapping function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# Check the resulting gene data
print(f"\nProcessed gene data shape: {gene_data.shape}")
print("First 10 genes in the processed data:")
print(gene_data.index[:10])


Gene mapping preview (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A

Processed gene data shape: (21278, 55)
First 10 genes in the processed data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (19845, 55)
First few genes with their expression values after normalization:
          GSM1612097  GSM1612098  GSM1612099  GSM1612100  GSM1612101  \
Gene                                                                   
A1BG             5.6        46.3        23.2        31.8        38.0   
A1BG-AS1         2.5        13.2         1.9         6.1        11.4   
A1CF            21.5        22.3        16.9        14.5        29.4   
A2M            370.0       516.4       627.4       659.6       732.6   
A2M-AS1         10.9        11.0         8.7        12.0        13.3   

          GSM1612102  GSM1612103  GSM1612104  GSM1612105  GSM1612106  ...  \
Gene                                                                  ...   
A1BG            25.0         7.4        10.4        37.4        15.0  ...   
A1BG-AS1         4.0         3.3         2.0         4.4         1.9  ...   
A1CF            43.7       135.8        15.9         5.5        34.2  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE65986.csv
Raw clinical data shape: (5, 56)
Clinical features:
                     GSM1612097  GSM1612098  GSM1612099  GSM1612100  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   
Age                        64.0        57.0        59.0        50.0   

                     GSM1612101  GSM1612102  GSM1612103  GSM1612104  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   
Age                        52.0        66.0        67.0        37.0   

                     GSM1612105  GSM1612106  ...  GSM1612142  GSM1612143  \
Endometrioid_Cancer         0.0         0.0  ...         0.0         0.0   
Age                        53.0        46.0  ...        69.0        49.0   

                     GSM1612144  GSM1612145  GSM1612146  GSM1612147  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   
Age                        74.0        56.0        5

Data shape after handling missing values: (55, 19847)
For the feature 'Endometrioid_Cancer', the least common label is '1.0' with 14 occurrences. This represents 25.45% of the dataset.
The distribution of the feature 'Endometrioid_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 49.0
  50% (Median): 57.0
  75%: 64.0
Min: 32.0
Max: 80.0
The distribution of the feature 'Age' in this dataset is fine.



Linked data saved to ../../output/preprocess/Endometrioid_Cancer/GSE65986.csv
